# Procesamiento Espacial

Esta libreta utiliza librerías de procesamiento de datos geoespaciales con dos objetivos:

1. Crear una relación de correspondencia entre códigos postales con su respectiva AGEB (Área Geoestadística Básica, definida por INEGI).
   
   En la mayoría de los casos, el código postal suele ser un área geográfica más grande que el AGEB. Por lo tanto, un código postal puede contener varias AGEB. En este caso, para definir qué AGEB corresponde a un código postal, se eligirá de forma aleatoria de entre el conjunto de AGEBS que intersectan a un código postal.
   
   Esto nos permitirá - en el futuro - poder relacionar nuestros datos con otras fuentes de datos que estén determinadas por AGEBs, la cual es de las unidades más comunes en estudios realizados en México. Por ejemplo, el Índice de Marginación Urbana realizada por CONAPO.

2. Calcular la distancia entre puntos, en este caso para calcular la distancia entre casa y escuela. 

*NOTA*: Se utilizan códigos postales y AGEBs del estado de Sonora.

### Importando librerías

In [5]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
import os
from geopy.distance import geodesic
import random

In [ ]:
# Definición de Rutas
cp_path = "../DatosExternos/CORREOSDEMEXICO/CP_SONORA_SHP"
ageb_path = "../DatosExternos/INEGI/MARCO_GEOESTADISTICO_SONORA_2024/conjunto_de_datos"
csv_path = "../data/processed"

# Definir sistema de coordenadas de referencia 
crs = "EPSG:6372" # Proyección optimizada para México

### Cargando polígonos de códigos postales (SEPOMEX)

In [17]:
# Cargar el SHP file de códigos postales
cp = gpd.read_file(os.path.join(cp_path, 'CP_26Son_v10.shp'))

# Reproyectar al mismo sistema de coordenadas
cp = cp.to_crs(crs)

# Corregir geometrías inválidas
cp["geometry"] = cp["geometry"].buffer(0)

# Definir campo ID 
cp_id = "d_codigo"

### Cargando polígonos de AGEBs (INEGI)

In [18]:
# Cargar el shapefile de AGEBs del INEGI
ageb = gpd.read_file(os.path.join(ageb_path, '26a.shp'))

# Reproyectar al mismo sistema de coordenadas
ageb = ageb.to_crs(crs)

# Corregir geometrías inválidas
ageb["geometry"] = ageb["geometry"].buffer(0)

# Definir campo ID
ageb_id = "CVEGEO"

### Crear tabla de correspondencia entre Código Postal - AGEB , mediante una intersección espacial (spatial join)

In [ ]:
# Filtrar solo el campo de la clave CVEGEO y la geometría
ageb = ageb[['CVEGEO', 'geometry']]

# Realizar la intersección espacial
union_cp_ageb = gpd.sjoin(cp, ageb, how="left", predicate="intersects")

# Función para elegir un AGEB aleatorio
def elegir_aleatorio(lista_agebs):
    valores = list(set(lista_agebs.dropna()))
    return random.choice(valores) if valores else None

# Agrupar por CP y elegir un solo AGEB al azar
df_cp_ageb = (
    union_cp_ageb.groupby(cp_id)[ageb_id]
    .apply(elegir_aleatorio)
    .reset_index()
)

# Guardar tabla CP-AGEB como CSV
df_cp_ageb.to_csv(os.path.join(csv_path, 'df_cp_ageb.csv'), index=False)


### Determinar AGEB a la que corresponde el Código Postal de cada punto

In [36]:
# Cargar el CSV 
df_indicador = pd.read_csv(os.path.join(csv_path, 'IndicadorObtenerBiblia_ing.csv'), dtype={"CDPOSTAL": str})

# Unir el DataFrame con la tabla de CP-AGEB
df_indicador = df_indicador.merge(df_cp_ageb, left_on='CDPOSTAL', right_on=cp_id, how='left')

df_indicador.drop(columns=['d_codigo'], inplace=True)

C:\Users\Janneth\AppData\Local\Temp\ipykernel_27828\1389732698.py:2: DtypeWarning: Columns (6,7,35,36,37,38,39,40,41,43) have mixed types. Specify dtype option on import or set low_memory=False.
  df_indicador = pd.read_csv(os.path.join(csv_path, 'IndicadorObtenerBiblia_ing.csv'), dtype={"CDPOSTAL": str})


### Calcular la distancia casa-escuela para cada punto

Utilizamos geopy ya que es la más recomendada para calcular distancias geográficas cuando se tienen coordenadas en términos de latitud y longitud. Además, geopy tiene en cuenta la curvatura de la Tierra, lo que la hace más precisa.

In [38]:
# Coordenadas del punto de referencia: Rectoría, Universidad de Sonora
unison_lat = 29.083496
unison_lon = -110.960289

In [49]:
# Crear geometría de puntos
df_indicador['geometry'] = [Point(xy) for xy in zip(df_indicador['LON'], df_indicador['LAT'])]

# Convertir a GeoDataFrame
gpdf_indicador = gpd.GeoDataFrame(df_indicador, geometry='geometry', crs=crs)  

# Corregir geometrías
gpdf_indicador['geometry'] = gpdf_indicador['geometry'].apply(lambda geom: geom if geom.is_valid else geom.buffer(0))

In [50]:
# Calcular distancia entre cada punto y el punto de referencia
def calcular_distancia(row):
    if row['geometry'].is_empty:
        return None
    else:
        return geodesic((unison_lat, unison_lon), (row['LAT'], row['LON'])).km # Distancia en kilómetros
    

# Aplicar la función de distancia a cada fila
gpdf_indicador['dist_casa_esc_km'] = gpdf_indicador.apply(calcular_distancia, axis=1)
gpdf_indicador['dist_casa_esc_km'] = gpdf_indicador['dist_casa_esc_km'].round(3)

In [54]:
# Filtrar las columnas necesarias
gpdf_indicador.drop(columns=['geometry'], inplace=True)

### Guardar CSV

In [55]:
# Guardar resultado en CSV
gpdf_indicador.to_csv(os.path.join(csv_path, 'IndicadorObtenerBiblia_ext.csv'), index=False)